In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
from sklearn.model_selection import train_test_split
from google.colab import files
import io

In [0]:
uploaded = files.upload()

Saving categoricals.csv to categoricals.csv


In [0]:
X = pd.read_csv(io.BytesIO(uploaded['x.csv']))
Y = X['worldwide_gross']
X = X.drop('worldwide_gross', axis = 1)
X = X.drop('gross', axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [0]:
from sklearn.preprocessing import StandardScaler

scaler =StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
scaler.mean_

array([3.34848760e+07, 2.00217414e+03, 2.13090157e+00, 1.08365560e+02,
       1.02316800e+04, 3.63641188e+07, 6.47020793e+00])

In [0]:
scaler.scale_

array([4.15973657e+07, 1.17387920e+01, 7.60904170e-01, 2.29622092e+01,
       1.99089329e+04, 4.45470651e+07, 1.06209395e+00])

In [0]:
scaler.transform(X_train)

array([[-0.0356964 ,  0.07035296, -0.36916813, ...,  1.66369138,
        -0.0755183 ,  0.40466484],
       [-0.56457604,  0.5814788 , -0.36916813, ..., -0.4166813 ,
         0.09184146, -1.19594686],
       [ 0.39702331,  0.32591588,  0.28794484, ...,  0.44785524,
         0.19385971,  1.06374024],
       ...,
       [-0.79896588,  0.24072824, -0.36916813, ..., -0.50754503,
         0.09184146,  0.31051121],
       [-0.7689159 ,  0.24072824, -0.61887106, ..., -0.47077762,
        -0.78263559,  0.21635758],
       [-0.79415789, -0.95189874, -0.36916813, ..., -0.49483717,
        -0.80620617, -0.53687146]])

In [0]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [0]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train, Y_train)
model_scaled.fit(X_train, Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
print(model.score(X_test, Y_test))
print(model_scaled.score(X_test, Y_test))

0.12576081489405178
0.12576081489405178


In [0]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),Lasso())

model_scaled.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [0]:
print(model_scaled.score(X_test, Y_test))

0.12576127907520807


In [0]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [0]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit(A)
transformer.transform(A)
##transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [0]:
X.shape

(4104, 7)

In [0]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 36)

In [0]:
model_poly = make_pipeline(PolynomialFeatures(2),Lasso())

model_poly.fit(X_train, Y_train)
model_poly.score(X_test, Y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.7254508941165533e+19, tolerance: 1.0181973457282362e+16
  positive)


-105.86134031054804

**ONE HOT - ENCODING**

In [0]:
d = pd.DataFrame([['Chile', 'Colombia', 'Colombia', 'Venezuela'],['hombre', 'mujer', 'hombre', 'mujer']])
d = d.T
d.columns = pd.Index(['pais', 'genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [0]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


In [0]:
movies_obj = pd.read_csv(io.BytesIO(uploaded['x.csv']))

In [0]:
movies_obj.apply(pd.Series.nunique).sort_values()

aspect_ratio                   21
imdb_score                     77
title_year                     89
duration.1                    168
budget                        388
production_budget             413
cast_total_facebook_likes    3456
gross                        3655
worldwide_gross              4015
dtype: int64

In [0]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.0MB/s 


In [0]:
categoricals = pd.read_csv(io.BytesIO(uploaded['categoricals.csv'])).set_index('Unnamed: 0')

In [0]:
categoricals.head(2)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [0]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [0]:
X_binenc = pd.concat([X, categoricals], axis=1)

In [0]:
X_binenc.head()

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,actor_1_name,director_name
0,425000000.0,2009.000000,1.780000,178.000000,4834.0,2.370000e+08,7.9,CCH Pounder,James Cameron
1,306000000.0,2002.130733,2.126976,108.577186,143.0,4.045539e+07,7.1,Doug Walker,Doug Walker
2,300000000.0,2007.000000,2.350000,169.000000,48350.0,3.000000e+08,7.1,Johnny Depp,Gore Verbinski
3,300000000.0,2015.000000,2.350000,148.000000,11700.0,2.450000e+08,6.8,Christoph Waltz,Sam Mendes
4,275000000.0,2012.000000,2.350000,164.000000,106759.0,2.500000e+08,8.5,Tom Hardy,Christopher Nolan


In [0]:
import category_encoders as ce 
encoder = ce.BinaryEncoder(cols = ['actor_1_name', 'director_name'])

In [0]:
encoder.fit_transform(X_binenc).shape

(4104, 31)

In [0]:
X_binenc = encoder.fit_transform(X_binenc)

In [0]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,Y)

In [0]:
X_train, X_test = (Xb_train[X.columns], Xb_test[X.columns])

In [0]:
model_binenc = Lasso()
model = Lasso()

In [0]:
model_binenc.fit(Xb_train, Y_train)
model.fit(X_train, Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
print(model_binenc.score(Xb_test, Y_test))
print(model.score(X_test, Y_test))

-0.008807153401767076
-0.006784918863646183
